<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-jhyeon-kim/blob/main/%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json

# Colab에 저장된 JSON 파일을 불러오기
with open('/content/2023_11_KICE.json', 'r', encoding='utf-8') as f:
    problems = json.load(f)

# 데이터 확인 (첫 번째 문제만 출력해보기)
print(problems[0])


{'id': '2023_11_KICE_1-3', 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 통

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# GPT-4 모델 로드
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # GPT-2로 대체
model = AutoModelForCausalLM.from_pretrained("gpt2")

# pad_token 설정을 위해 추가적으로 토크나이저를 수정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰이 설정되지 않으면 eos_token으로 설정

def prediction(problem):
    """
    문제 데이터를 입력으로 받아 GPT 모델로 예측하는 함수.
    문제 텍스트를 기반으로 정답 예측.
    """
    # 문제의 지문 추출
    context = problem['paragraph']

    # 문제는 'problems' 리스트에 들어 있음
    for sub_problem in problem['problems']:
        # 해당 문제의 질문과 선택지 추출
        question = sub_problem['question']
        choices = sub_problem['choices']

        # 문제 텍스트를 GPT 입력 형식으로 변환
        input_text = f"{context}\n{question}\n선택지: {', '.join(choices)}\n답은?"

        # 모델 입력을 위한 토크나이징 (truncation=True와 max_length=512로 설정)
        inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512, padding=True)

        # 모델이 있는 디바이스로 텐서 이동 (GPU로 이동)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = {key: value.to(device) for key, value in inputs.items()}
        model.to(device)

        # 모델을 통해 예측 수행 (attention_mask와 pad_token_id 설정)
        outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=50,
            pad_token_id=tokenizer.pad_token_id  # 이제 pad_token_id가 설정됨
        )

        # 생성된 텍스트 디코딩
        prediction_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # GPT가 생성한 답을 반환
        return prediction_text



In [27]:
from difflib import SequenceMatcher

def find_best_match(gpt_prediction, choices):
    """
    GPT가 생성한 답과 선택지 중 가장 유사한 선택지를 찾는 함수.
    """
    best_match = None
    best_ratio = 0

    for choice in choices:
        # GPT의 생성된 텍스트와 선택지 간의 유사도 비교
        ratio = SequenceMatcher(None, gpt_prediction, choice).ratio()

        if ratio > best_ratio:
            best_match = choice
            best_ratio = ratio

    return best_match

def evaluate_model(problems):
    correct = 0  # 맞춘 문제 수
    total = len(problems)  # 전체 문제 수

    for problem in problems:
        # 문제의 지문 추출 (paragraph를 기반으로 예측)
        context = problem['paragraph']

        # 각 문제들에 접근 ('problems' 필드 안에 있는 문제들)
        for sub_problem in problem['problems']:
            # 문제의 질문과 선택지 추출
            question = sub_problem['question']
            choices = sub_problem['choices']

            # GPT 예측 결과
            gpt_prediction = prediction(problem)

            # GPT가 생성한 답에서 가장 유사한 선택지 찾기
            predicted_choice = find_best_match(gpt_prediction, choices)

            # 정답 확인 (정답 인덱스는 1부터 시작하므로 -1)
            if predicted_choice == choices[sub_problem['answer'] - 1]:
                correct += 1

    # 점수 계산
    score = (correct / total) * 100
    print(f"GPT-4 모델의 최종 점수: {score}%")
    return score


In [28]:
evaluate_model(problems)

GPT-4 모델의 최종 점수: 72.72727272727273%


72.72727272727273